In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, classification_report
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

In [122]:
us_accidents = pd.read_csv("C:/Users/Hobbs/OneDrive/Documents/SpartaHack2026/ShouldDrive_Spartahack11/US_Accidents_March23.csv")

In [123]:
us_accidents.head()


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [124]:
michigan = us_accidents.loc[us_accidents['State'] == 'MI'].copy()


In [125]:
michigan["Severity"] = pd.to_numeric(michigan["Severity"], errors="coerce")

conditions = [
    michigan["Severity"] == 1,
    michigan["Severity"] == 2,
    michigan["Severity"].isin([3, 4])
]

choices = ["Low", "Medium", "High"]

michigan["Risk_Level"] = np.select(
    conditions,
    choices,
    default="Unknown"   # must be a string
)


In [126]:
michigan.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Risk_Level'],
      dtype='object')

In [127]:
model_cols = [
    "Risk_Level",
    "Temperature(F)",
    "Wind_Chill(F)",
    "Humidity(%)",
    "Pressure(in)",
    "Visibility(mi)",
    "Wind_Direction",
    "Wind_Speed(mph)",
    "Precipitation(in)",
    "Weather_Condition",
    "Sunrise_Sunset"
]

michigan_model = michigan[model_cols].copy()


In [128]:
michigan_model = michigan_model.dropna()

In [129]:
michigan_model.describe(include = 'all')

,Risk_Level,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Sunrise_Sunset
count,97315,97315.000000,97315.000000,97315.000000,97315.000000,97315.000000,97315,97315.000000,97315.000000,97315,97315
unique,3,NaN,NaN,NaN,NaN,NaN,23,NaN,NaN,68,2
top,Medium,NaN,NaN,NaN,NaN,NaN,W,NaN,NaN,Fair,Day
freq,74580,NaN,NaN,NaN,NaN,NaN,9854,NaN,NaN,28831,62710
mean,NaN,47.284649,43.056608,71.474603,29.259221,8.082903,NaN,9.238104,0.006383,NaN,NaN
std,NaN,21.130868,25.184739,18.121675,0.351208,3.168320,NaN,5.352295,0.032206,NaN,NaN
min,NaN,-19.000000,-35.600000,5.000000,27.650000,0.000000,NaN,0.000000,0.000000,NaN,NaN
25%,NaN,30.900000,23.000000,59.000000,29.040000,7.000000,NaN,6.000000,0.000000,NaN,NaN
50%,NaN,44.000000,40.000000,75.000000,29.210000,10.000000,NaN,9.000000,0.000000,NaN,NaN
75%,NaN,66.000000,66.000000,86.000000,29.420000,10.000000,NaN,13.000000,0.000000,NaN,NaN


In [130]:
michigan_model["Weather_Condition"].unique()

array(['Overcast', 'Light Rain', 'Light Snow', 'Light Drizzle', 'Snow',
       'Rain', 'Heavy Snow', 'Mostly Cloudy', 'Haze', 'Scattered Clouds',
       'Light Freezing Rain', 'Light Freezing Drizzle', 'Partly Cloudy',
       'Heavy Rain', 'Heavy Thunderstorms and Rain', 'Light Ice Pellets',
       'Fog', 'Light Rain Showers', 'Clear', 'Thunderstorms and Rain',
       'Light Thunderstorms and Rain', 'Fair', 'Cloudy', 'Patches of Fog',
       'Shallow Fog', 'Light Rain with Thunder', 'Thunder', 'T-Storm',
       'Heavy T-Storm', 'Showers in the Vicinity',
       'Partly Cloudy / Windy', 'Fair / Windy', 'Thunder in the Vicinity',
       'Rain / Windy', 'Cloudy / Windy', 'Mostly Cloudy / Windy',
       'Light Rain / Windy', 'Wintry Mix', 'Light Snow / Windy',
       'Haze / Windy', 'Blowing Snow / Windy', 'N/A Precipitation',
       'Sleet', 'Thunder / Wintry Mix', 'Mist', 'Drizzle',
       'Sleet / Windy', 'Snow / Windy', 'Heavy Rain / Windy',
       'Light Drizzle / Windy', 'T-Storm / W

In [131]:
w = (
    michigan_model["Weather_Condition"]
    .astype(str)          # protects against NaN
    .str.lower()
)

conditions = [
    w.str.contains(r"thunder|t[-\s]?storm|hail", regex=True),
    w.str.contains(r"snow|sleet|ice|freezing|wintry", regex=True),
    w.str.contains(r"rain|drizzle|shower", regex=True),
    w.str.contains(r"fog|mist|haze", regex=True),
    w.str.contains(r"cloud|overcast", regex=True),
    w.str.contains(r"clear|fair", regex=True),
]

choices = [
    "Storm",
    "Snow_Ice",
    "Rain",
    "Fog_Haze",
    "Cloudy",
    "Clear",
]

michigan_model["Weather_Simple"] = np.select(
    conditions,
    choices,
    default="Other"
)

michigan_model = michigan_model.drop(columns="Weather_Condition")


In [132]:
michigan_model.to_csv("michigan.csv")

In [133]:
michigan_model.groupby(["Risk_Level"]).count()

,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Sunrise_Sunset,Weather_Simple
Risk_Level,,,,,,,,,,
High,21712,21712,21712,21712,21712,21712,21712,21712,21712,21712
Low,1023,1023,1023,1023,1023,1023,1023,1023,1023,1023
Medium,74580,74580,74580,74580,74580,74580,74580,74580,74580,74580


In [134]:
michigan_model.describe(include='all')

,Risk_Level,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Sunrise_Sunset,Weather_Simple
count,97315,97315.000000,97315.000000,97315.000000,97315.000000,97315.000000,97315,97315.000000,97315.000000,97315,97315
unique,3,NaN,NaN,NaN,NaN,NaN,23,NaN,NaN,2,7
top,Medium,NaN,NaN,NaN,NaN,NaN,W,NaN,NaN,Day,Cloudy
freq,74580,NaN,NaN,NaN,NaN,NaN,9854,NaN,NaN,62710,41418
mean,NaN,47.284649,43.056608,71.474603,29.259221,8.082903,NaN,9.238104,0.006383,NaN,NaN
std,NaN,21.130868,25.184739,18.121675,0.351208,3.168320,NaN,5.352295,0.032206,NaN,NaN
min,NaN,-19.000000,-35.600000,5.000000,27.650000,0.000000,NaN,0.000000,0.000000,NaN,NaN
25%,NaN,30.900000,23.000000,59.000000,29.040000,7.000000,NaN,6.000000,0.000000,NaN,NaN
50%,NaN,44.000000,40.000000,75.000000,29.210000,10.000000,NaN,9.000000,0.000000,NaN,NaN
75%,NaN,66.000000,66.000000,86.000000,29.420000,10.000000,NaN,13.000000,0.000000,NaN,NaN


In [135]:
# Features and target
X = michigan_model.drop(columns=["Risk_Level"])
y = michigan_model["Risk_Level"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


print("Train size:", X_train.shape)
print("Test size:", X_test.shape)


Train size: (77852, 10)
Test size: (19463, 10)


In [136]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_cols = X.select_dtypes(include=["object", "category"]).columns
numeric_cols = X.select_dtypes(exclude=["object", "category"]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)


In [137]:
over = RandomOverSampler(
    sampling_strategy={"Low": 15000},
    random_state=42
)

under = RandomUnderSampler(
    sampling_strategy={"Medium": 35000},
    random_state=42
)

rf = RandomForestClassifier(
    n_estimators=300,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1,
    class_weight=None  # usually remove if you resample
)

model = ImbPipeline(steps=[
    ("preprocess", preprocessor),
    ("over", over),
    ("under", under),
    ("rf", rf)
])

model.fit(X_train, y_train)


              precision    recall  f1-score   support

        High       0.51      0.17      0.26      4342
         Low       0.13      0.45      0.20       205
      Medium       0.79      0.92      0.85     14916

    accuracy                           0.74     19463
   macro avg       0.48      0.51      0.43     19463
weighted avg       0.72      0.74      0.71     19463

[[  755    98  3489]
 [    1    92   112]
 [  720   544 13652]]


In [138]:
y_pred = model.predict(X_test)


print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

        High       0.51      0.17      0.26      4342
         Low       0.13      0.45      0.20       205
      Medium       0.79      0.92      0.85     14916

    accuracy                           0.74     19463
   macro avg       0.48      0.51      0.43     19463
weighted avg       0.72      0.74      0.71     19463

[[  755    98  3489]
 [    1    92   112]
 [  720   544 13652]]


In [139]:
labels = model.named_steps["rf"].classes_

cm = confusion_matrix(
    y_test,
    y_pred,
    labels=labels
)

cm_df = pd.DataFrame(
    cm,
    index=[f"Actual: {l}" for l in labels],
    columns=[f"Predicted: {l}" for l in labels]
)

display(cm_df)

,Predicted: High,Predicted: Low,Predicted: Medium
Actual: High,755,98,3489
Actual: Low,1,92,112
Actual: Medium,720,544,13652


In [140]:
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm,
    index=["Actual Low", "Actual High", "Actual Medium"],
    columns=["Predicted Low", "Predicted High", "Predicted Medium"]
)

cm_df

,Predicted Low,Predicted High,Predicted Medium
Actual Low,755,98,3489
Actual High,1,92,112
Actual Medium,720,544,13652


In [141]:
sample_input = X_test.iloc[[0]]

probs = model.predict_proba(sample_input)[0]
classes = model.classes_

Risk_Level_output = dict(zip(classes, probs))
predicted_risk = classes[probs.argmax()]

print("Predicted risk:", predicted_risk)
print("Real Risk:", y_test.iloc[0])


Predicted risk: Medium
Real Risk: Medium


In [142]:
labels = ["Low", "Medium", "High"]

# clean strings (case + whitespace)
y_train_clean = pd.Series(y_train).astype(str).str.strip().str.title()
y_test_clean  = pd.Series(y_test).astype(str).str.strip().str.title()

# keep only the 3 target classes (drop Unknown/other)
train_mask = y_train_clean.isin(labels)
test_mask  = y_test_clean.isin(labels)

y_train_clean = y_train_clean[train_mask]
X_train_clean = X_train.loc[y_train_clean.index]

y_test_clean = y_test_clean[test_mask]
X_test_clean = X_test.loc[y_test_clean.index]

to_int = {"Low": 0, "Medium": 1, "High": 2}
to_lab = {0: "Low", 1: "Medium", 2: "High"}

y_train_i = y_train_clean.map(to_int)
y_test_i  = y_test_clean.map(to_int)


In [146]:
over = RandomOverSampler(
    sampling_strategy={
        to_int["Low"]: 8000,
        to_int["High"]: 20000
    },
        random_state=42
)

under = RandomUnderSampler(
    sampling_strategy={
        to_int["Medium"]: 25000}, random_state=42)

xgb = XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    n_estimators=600,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_weight=5,
    reg_lambda=1.0,
    eval_metric="mlogloss",
    random_state=42,
    n_jobs=-1
)

model = ImbPipeline(steps=[
    ("preprocess", preprocessor),
    ("over", over),
    ("under", under),
    ("xgb", xgb)
])

model.fit(X_train, y_train_i)

y_pred_i = model.predict(X_test)
y_pred = pd.Series(y_pred_i).map(to_lab)   # back to Low/Medium/High


In [147]:
# predictions already mapped to strings
y_pred = pd.Series(y_pred_i).map(to_lab)

# map y_test to strings in the same way
y_test_s = pd.Series(y_test_i).map(to_lab)   # <-- use y_test_i, not y_test

from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

print(classification_report(y_test_s, y_pred, labels=labels))

cm = confusion_matrix(y_test_s, y_pred, labels=labels)
cm_df = pd.DataFrame(
    cm,
    index=[f"Actual: {c}" for c in labels],
    columns=[f"Predicted: {c}" for c in labels]
)
display(cm_df)


              precision    recall  f1-score   support

         Low       0.08      0.53      0.14       205
      Medium       0.82      0.73      0.77     14916
        High       0.37      0.41      0.39      4342

    accuracy                           0.65     19463
   macro avg       0.42      0.56      0.43     19463
weighted avg       0.71      0.65      0.68     19463



,Predicted: Low,Predicted: Medium,Predicted: High
Actual: Low,109,80,16
Actual: Medium,1072,10825,3019
Actual: High,217,2325,1800
